<a href="https://colab.research.google.com/github/10udCryp7/TV-command-synthesis/blob/main/src_prototype/Phase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# F5-TTS
!pip install -q f5-tts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 78.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
# Reference Speech
!gdown 1uTreohCIiYSlrQTa3fuH1_IQjdeW1SaE --quiet
# Reference Meta (including Text)
!gdown 1PBs6r3cqhFxWzy9s5wrGpF6-5ZnoOWxE --quiet

!unzip -q audio.zip -d audio
!unzip -q json.zip -d json

In [2]:
import os
import json
import soundfile as sf
import pandas as pd
import random
import ast
import numpy as np
import uuid
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp
from pathlib import Path
import time
from typing import Any, List
from importlib.resources import files
from f5_tts.api import F5TTS

class ReferenceCache:
    def __init__(self, audio_folder_path: str, json_folder_path: str):
        self.audio_folder_path = Path(audio_folder_path)
        self.json_folder_path = Path(json_folder_path)
        self._audio_files = None
        self._json_files = None
        self._ref_texts = {}

    @property
    def audio_files(self):
        if self._audio_files is None:
            self._audio_files = [f for f in self.audio_folder_path.iterdir() if f.is_file()]
        return self._audio_files

    @property
    def json_files(self):
        if self._json_files is None:
            self._json_files = [f for f in self.json_folder_path.iterdir() if f.suffix == '.json']
        return self._json_files

    def get_ref_text(self, json_path: Path) -> str:
        if str(json_path) not in self._ref_texts:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self._ref_texts[str(json_path)] = data.get('text', '')
        return self._ref_texts[str(json_path)]

    def sample_reference(self):
        audio_file = random.choice(self.audio_files)
        json_file = random.choice(self.json_files)
        ref_id = json_file.stem
        ref_speech_path = str(audio_file)
        ref_text = self.get_ref_text(json_file)
        return ref_id, ref_speech_path, ref_text

In [3]:
import os
import json
import soundfile as sf
import pandas as pd
import random
import ast
import numpy as np
import uuid
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp
from pathlib import Path
import time
from typing import Any, List
from importlib.resources import files
from f5_tts.api import F5TTS

class SpeechSynthesis:
  def __init__(self, ref_folder_path, json_folder_path, num_workers):
    self.ref_folder_path = ref_folder_path
    self.json_folder_path = json_folder_path
    self.num_workers = num_workers

  def load_model(self):
    f5tts = F5TTS(device = 'cuda')
    return f5tts


  def audio_generate(self, command: str, model, ref_file: str = "", ref_text: str = ""):
      wav, sr, spec = model.infer(
          ref_file=ref_file,
          ref_text=ref_text,
          gen_text=command,
          seed=None,
      )
      return wav, sr, spec

  def process_single_command(self, row: pd.Series, ref_cache: ReferenceCache, export_path: str, model: Any):
    try:
        ref_id, ref_speech_path, ref_text = ref_cache.sample_reference()
        wav, sr, _ = self.audio_generate(row['text'], model, ref_speech_path, ref_text)

        cur_id = row['id']
        cmd_path = Path(export_path) / f"{row['type']}_{cur_id}"
        cmd_path.mkdir(parents=True, exist_ok=True)

        # Save full audio
        sf.write(cmd_path / f"{row['type']}_{cur_id}_full.wav", wav, sr)

        type_segments = {}
        text_segments = {}

        if pd.notna(row['segments']) and row['segments'] not in [None, "", "nan"]:
            seg_list = ast.literal_eval(row['segments'])
            for count, seg in enumerate(seg_list):
                key = list(seg.keys())[0]
                seg_wav, seg_sr, _ = self.audio_generate(seg[key], model, ref_speech_path, ref_text)
                sf.write(cmd_path / f"{row['type']}_{cur_id}_seg_{count}.wav", seg_wav, seg_sr)
                list_key = list(seg.keys())
                type_segments[list_key[0]] = seg[list_key[1]] if len(list_key) > 1 else ""
                text_segments[list_key[0]] = seg[list_key[0]]

        json_template = {
            "id": cur_id,
            "type": row['type'],
            "command": row['text'],
            "sampling_rate": sr,
            "num_segments": len(type_segments),
            "type_segments": type_segments,
            "text_segments": text_segments,
            "ref_id": ref_id,
            "ref_file": ref_speech_path,
            "ref_text": ref_text
        }
        with open(cmd_path / f"{row['type']}_{cur_id}.json", 'w', encoding='utf-8') as f:
            json.dump(json_template, f, ensure_ascii=False, indent=2)

        return json_template
    except Exception as e:
        print(f"Failed to process: {row['text']}, error: {e}")
        return None


  def worker_task(self, rows: List[pd.Series], ref_folder: str, json_folder: str, export_path: str):
      # Mỗi worker load model riêng
      model = self.load_model()
      ref_cache = ReferenceCache(ref_folder, json_folder)
      results = []
      for row in rows:
          res = self.process_single_command(row, ref_cache, export_path, model)
          if res:
              results.append(res)
      return results

  def command_synthesis_pipeline_multi_model(
    self,
    text_command,
    export_path
  ):
    start_time = time.time()
    os.makedirs(export_path, exist_ok=True)

    if self.num_workers is None:
        self.num_workers = min(mp.cpu_count(), 4)  # Giới hạn số model load

    # Chia dữ liệu thành N phần, mỗi worker xử lý riêng
    chunks = np.array_split(text_command, self.num_workers)

    all_results = []
    with ThreadPoolExecutor(max_workers=self.num_workers) as executor:
        futures = [
            executor.submit(self.worker_task, chunk.to_dict('records'), self.ref_folder_path, self.json_folder_path, export_path)
            for chunk in chunks
        ]
        for i, future in enumerate(futures):
            res_list = future.result()
            all_results.extend(res_list)
            print(f"Worker {i+1} done ({len(res_list)} items)")

    print(f"Pipeline finished in {time.time() - start_time:.2f}s")
    return pd.DataFrame(all_results)



In [4]:
import gdown

gid = '1StjZP25ROcexajtVHml-9LkfECkVsb3s'
url = f"https://drive.google.com/uc?id={gid}"

output = gdown.download(url, quiet=False)  # output sẽ là tên file đã tải

Downloading...
From: https://drive.google.com/uc?id=1StjZP25ROcexajtVHml-9LkfECkVsb3s
To: /content/sample-5000-100.zip
100%|██████████| 216k/216k [00:00<00:00, 112MB/s]


In [5]:
!unzip -q {output} -d "text-command"

['output_parts']

In [13]:
import shutil

param = {
    'ref_folder_path': "audio/dung",
    'json_folder_path': "json/dung",
    'num_workers': 4,
}

csv_folder = 'text-command/output_parts'
export_folder = 'synthesis_command'

def main(export_folder):
  os.makedirs(export_folder, exist_ok=True)
  list_speech_df = []
  speech_synthesis = SpeechSynthesis(**param)
  for csv_file in os.listdir(csv_folder):
    if csv_file.endswith('.csv'):
      csv_path = os.path.join(csv_folder, csv_file)
      df = pd.read_csv(csv_path)
    else:
      continue

    name = os.path.splitext(csv_file)[0]
    export_path = f'{export_folder}/synthesis_command_{name}'
    os.makedirs(export_path, exist_ok=True)
    df_res = speech_synthesis.command_synthesis_pipeline_multi_model(df, export_path=export_path)
    shutil.make_archive(export_path, 'zip', export_path)
    list_speech_df.append(df_res)

    df_path = os.path.join(export_path, f'synthesis_command_{name}.csv')
    df.to_csv(df_path, index=False)

  list_speech_df = pd.concat(list_speech_df)
  list_speech_df.to_csv('synthesis_command.csv', index=False)

In [ ]:
if __name__ == '__main__':
  main(export_folder = export_folder)

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Download Vocos from huggingface charactr/vocos-mel-24khz
Download Vocos from huggingface charactr/vocos-mel-24khz
Download Vocos from huggingface charactr/vocos-mel-24khz
Download Vocos from huggingface charactr/vocos-mel-24khz


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.yaml:   0%|          | 0.00/461 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.4M [00:00<?, ?B/s]

...




100%|██████████| 1/1 [00:10<00:00, 10.52s/it]

100%|██████████| 1/1 [00:10<00:00, 10.51s/it]


100%|██████████| 1/1 [00:10<00:00, 10.50s/it]


Converting audio...
Using cached preprocessed reference audio...
Using custom reference text...

ref_text    Particularly among millennials, and I know millennials probably hate being called millennials, I'm sorry, whatever, but the fact. 
gen_text 0 I can’t believe she said that! Switch to AV mode and go back to the previous episode.


Generating audio in 1 batches...
Converting audio...
Converting audio...
Using cached preprocessed reference audio...
Using custom reference text...

ref_text    Uhm, the dislocated right shoulder and all that stuff that, you know, I thought I may never play again. So that kind of put me on the fast track to a degenerative shoulder and all kinds of arthritic changes and stuff like that. So. 
gen_text 0 set volume to 20.


Generating audio in 1 batches...
Using cached preprocessed reference audio...
Using custom reference text...

ref_text    Well sure, but that was so we could be a lawyer. Okay, so now we're a lawyer. Come on, I'll shoot you for a dolla

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:10<00:00, 10.48s/it]


Converting audio...
Using cached preprocessed reference audio...
Using custom reference text...

ref_text    I like the video introduction. It's a great way to get started on the topic. 
gen_text 0 and can you believe how fast time flies?


Generating audio in 1 batches...





  0%|          | 0/1 [00:00<?, ?it/s]

In [1]:
!zip synthesis_command_0.zip /content/synthesis_command -r

  adding: content/synthesis_command/ (stored 0%)
  adding: content/synthesis_command/synthesis_command_part_28/ (stored 0%)
  adding: content/synthesis_command/synthesis_command_part_28/single_active_d6772e20/ (stored 0%)
  adding: content/synthesis_command/synthesis_command_part_28/single_active_d6772e20/single_active_d6772e20_full.wav (deflated 40%)
  adding: content/synthesis_command/synthesis_command_part_28/single_active_d6772e20/single_active_d6772e20.json (deflated 41%)
  adding: content/synthesis_command/synthesis_command_part_28/non_active_868e673a/ (stored 0%)
  adding: content/synthesis_command/synthesis_command_part_28/non_active_868e673a/non_active_868e673a_full.wav (deflated 22%)
  adding: content/synthesis_command/synthesis_command_part_28/non_active_868e673a/non_active_868e673a.json (deflated 38%)
  adding: content/synthesis_command/synthesis_command_part_28/single_active_e3c5c9ba/ (stored 0%)
  adding: content/synthesis_command/synthesis_command_part_28/single_active_e